##### 1단계: LangChain 설치 및 설정

gcloud cli 인증


In [1]:
#gcloud auth application-default login

##### 2단계: Groq Qwen2.5 32B 설치 및 설정 지침  -> 대신 GCP gemini로 변경

In [2]:
import os
from langchain_google_vertexai import ChatVertexAI


# Vertex AI Gemini 기반 LLM
llm = ChatVertexAI(
    model="gemini-2.5-flash",   # 또는 "gemini-2.5-pro"
    temperature=0.0,
    
)


##### 3단계: NVIDIA bge-m3 설치 및 설정 -> 대신 vertex 임베딩

In [ ]:
#pip install -qU langchain-nvidia-ai-endpoints

''

In [4]:

from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

##### 4단계: Milvus 설치 및 설정 -> 대신 faiss

##### 5단계: RAG 챗봇 구축

In [5]:
import os
import json
import faiss

from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.prompts import ChatPromptTemplate


FAISS_INDEX_PATH = "paper.faiss"
DATA_JSON_PATH   = "paper.json"
MAP_JSON_PATH    = "paper_to_model.json"


def load_vector_store(
    index_path: str = FAISS_INDEX_PATH,
    data_path: str = DATA_JSON_PATH,
    map_path: str = MAP_JSON_PATH,
):
    print(f"[INFO] Loading FAISS index '{index_path}'...")
    index = faiss.read_index(index_path)

    print(f"[INFO] Loading paper data '{data_path}' and map '{map_path}'...")
    with open(data_path, "r", encoding="utf-8") as f:
        paper_DATA = json.load(f)
    with open(map_path, "r", encoding="utf-8") as f:
        index_to_model_map = json.load(f)

    if not index_to_model_map:
        raise ValueError("index_to_model_map is empty. Run build_database.py first.")

    print("[INFO] Rebuilding LangChain docstore from FAISS map...")
    docstore_dict = {}
    index_to_docstore_id = {}

    for str_idx, model_name in index_to_model_map.items():
        int_idx = int(str_idx)
        info = paper_DATA.get(model_name)
        if info is None:
            raise KeyError(f"paper.json is missing entry for '{model_name}'.")

        doc_id = model_name
        text = f"model: {model_name}. " + " ".join(
            f"{key}: {value}" for key, value in info.items() if value is not None
        )

        metadata = {"model_name": model_name, **info}
        doc = Document(page_content=text, metadata=metadata)

        docstore_dict[doc_id] = doc
        index_to_docstore_id[int_idx] = doc_id

    if index.ntotal != len(index_to_docstore_id):
        print(f"[WARN] Index size ({index.ntotal}) != mapping count ({len(index_to_docstore_id)}).")

    docstore = InMemoryDocstore(docstore_dict)

    print("[INFO] Building LangChain FAISS VectorStore...")
    return FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
    )


vector_store = None  

try:
    vector_store = load_vector_store()
    print("[INFO] FAISS index loaded and reconstructed.")

except FileNotFoundError as e:
    print(f"[ERROR] Missing required file: {e}")
    print("Check paper.faiss, paper.json, paper_to_model.json paths.")
except Exception as e:
    import traceback
    print(f"[ERROR] Exception while loading FAISS: {e}")
    traceback.print_exc()


[INFO] FAISS 인덱스 'paper.faiss' 로드 중...
[INFO] 원본 데이터 'paper.json' 및 맵 'paper_to_model.json' 로드 중...
[INFO] LangChain Docstore 재구성 중...
[INFO] LangChain FAISS VectorStore 생성 중...
[INFO] FAISS 인덱스 수동 로드 및 재구성 완료


프롬프트 정의

In [6]:


print("[INFO] RAG 프롬프트 정의 중...")

key_descriptions = """
[데이터 필드 설명]
- title: 논문의 제목
- author: 저자 목록
- year: 논문의 출판 연도 혹은 ArXiv 게시 연도 (최초 공개 시점)
- keywords: 논문의 핵심 내용 요약
- tasks: 논문에서 해결하고자 하는 주요 과업(Task) 정의
- abstract: 논문의 초록 (연구의 목적, 방법, 결과 요약)
- related work: 논문이 참고하거나 비교한 기존 연구들에 대한 설명 (기존 연구와의 차이점 등)
- Experiments: 실험 내용 (데이터셋, 실험 환경, 성능 평가 결과 등)
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 논문/모델 정보를 설명하는 어시스턴트입니다.\n"
            f"제공되는 정보는 다음과 같은 구조로 이루어져 있습니다:\n{key_descriptions}\n"
            "주어진 context(논문 메타데이터와 설명)에 근거해서만 답변하세요. "
            "모르면 모른다고 말하세요."
        ),
        (
            "human",
            "질문: {question}\n\n"
            "다음은 검색된 논문/모델 정보입니다:\n"
            "{context}\n\n"
            "위의 데이터 필드 설명과 정보를 바탕으로 한국어로 자세히 답변해 주세요."
        ),
    ]
)

print("[INFO] 프롬프트 설정 완료")


[INFO] RAG 프롬프트 정의 중...
[INFO] 프롬프트 설정 완료


In [7]:

from datetime import datetime

TOP_TIER_VENUES = {"NEURIPS", "ICML", "ICLR", "CVPR", "ACL", "NAACL", "EMNLP", "ECCV"}
DEFAULT_YEAR_WINDOW = 5
RECENCY_WEIGHT = 0.1 #최신 논문 가중치 
VENUE_WEIGHT = 0.3 #탑티어 학회 점수 가중치


# 사용자의 선호 연도와 선호 학회를 입력 받음.
def collect_user_preferences():
    print("[SETUP] Enter your search preferences.")
    #최근 몇 년 논문에 집중할지? (숫자 입력, 기본값 5)
    year_raw = input("최근 몇 년 논문을 우선할까요? (숫자 입력, 기본값 5):: ").strip()
    try:
        year_window = int(year_raw) if year_raw else DEFAULT_YEAR_WINDOW
    except ValueError:
        print("[경고] 숫자가 아닙니다. 기본값 5년을 사용합니다.")
        year_window = DEFAULT_YEAR_WINDOW

    #학회 우선순위를 어떻게 둘지?
    venue_raw = input("학회 우선순위를 선택하세요. (top/top-only/none or comma list, e.g., NeurIPS,ICML): ").strip()

    venue_mode = "all"
    preferred = []
    if venue_raw:
        lower = venue_raw.lower()
        if lower in ("top", "top-tier", "top priority"): # 탑티어에 점수 부여 + 다른 학회도 봄 
            venue_mode = "top"
        elif lower in ("top-only", "top tier only", "top-tier only"): # 탑티어만 
            venue_mode = "top_only"
        else:
            venue_mode = "custom" # 직접 지정
            preferred = [v.strip().upper() for v in venue_raw.split(",") if v.strip()]

    return {
        "year_window": year_window, # 최근 몇 년 논문에 집중할지(default 5년)
        "venue_mode": venue_mode, # top-tier 우선, top-tier only, custom venue, 전체 
        "preferred_venues": preferred, # 사용자가 직접 지정한 venue 리스트
    }


#사용자의 선호 설정 문자열로 요약 (dict -> str)
def describe_preferences(preferences: dict) -> str:
    parts = [f"recent {preferences.get('year_window', DEFAULT_YEAR_WINDOW)} years"]
    venue_mode = preferences.get("venue_mode", "all")
    if venue_mode == "top":
        parts.append("top-tier prioritized")
    elif venue_mode == "top_only":
        parts.append("top-tier only")
    elif venue_mode == "custom":
        prefs = preferences.get("preferred_venues", [])
        if prefs:
            parts.append("venues: " + ", ".join(prefs))
    else:
        parts.append("all venues")
    return " | ".join(parts)


def rerank_with_preferences(docs_with_scores, preferences: dict):
    # 검색 결과(docs_with_scores)에 선호도 기반 가중치를 더해 재정렬.
    preferences = preferences or {}
    year_window = preferences.get("year_window", DEFAULT_YEAR_WINDOW)
    venue_mode = preferences.get("venue_mode", "all")
    preferred = {v.upper() for v in preferences.get("preferred_venues", [])}
    current_year = datetime.now().year

    applied = {"year_cutoff": None, "venue_mode": venue_mode}
    cutoff = current_year - year_window if year_window else None

    filtered = docs_with_scores
    
    #연도 필터링 
    if cutoff is not None:
        #필터링 후 결과가 비면 필터링을 포기하고 원래 목록을 그대로 사용
        filtered = [ 
            (doc, score)
            for doc, score in filtered
            if int(doc.metadata.get("year", 0) or 0) >= cutoff
        ]
        applied["year_cutoff"] = cutoff
        if not filtered: 
            filtered = docs_with_scores
            applied["year_cutoff"] = None

    reranked = []
    for doc, score in filtered:
        meta = doc.metadata or {}
        #선호도 기반 점수 재계산
        base_similarity = -float(score) 
        #최신 연도 가중치
        year = meta.get("year")
        recency_bonus = 0.0
        if isinstance(year, (int, float)):
            years_old = max(current_year - int(year), 0)
            if year_window:
                recency_bonus = max(0, year_window - years_old) * RECENCY_WEIGHT
        #학회 가중치
        venue = str(meta.get("venue", "")).upper()
        venue_bonus = 0.0
        if venue_mode == "top":
            if venue in TOP_TIER_VENUES:
                venue_bonus = VENUE_WEIGHT
        elif venue_mode == "top_only":
            if venue in TOP_TIER_VENUES:
                venue_bonus = VENUE_WEIGHT
            else:
                continue # top-tier 아니면 아예 제외
        elif venue_mode == "custom":
            if venue in preferred:
                venue_bonus = VENUE_WEIGHT
        #최종 점수
        final_score = base_similarity + recency_bonus + venue_bonus
        reranked.append((doc, final_score))
    #점수가 높은 순으로 재정렬.
    if not reranked:
        reranked = [(doc, -float(score)) for doc, score in docs_with_scores]

    reranked.sort(key=lambda x: x[1], reverse=True)
    return reranked, applied


(2) RAG 한 번 호출하는 함수만 두기

In [10]:

# ===============================
# RAG 함수 정의
# ===============================

def rag_answer(question: str, k: int = 4, preferences: dict | None = None):
    """FAISS에서 관련 문서를 검색하고, LLM으로 답변을 생성하는 단일 RAG 함수."""
    if vector_store is None:
        raise RuntimeError("vector_store가 초기화되지 않았습니다. FAISS 로드 셀을 먼저 확인하세요.")

    print(f"[RAG] 검색 질의: {question}")
    docs_with_scores = vector_store.similarity_search_with_score(question, k=k)
    print(f"[RAG] 검색된 문서 수: {len(docs_with_scores)}")

    #연도/학회 규칙 기반 재정렬
    reranked_docs, applied_filters = rerank_with_preferences(docs_with_scores, preferences or {})
    docs = [doc for doc, _ in reranked_docs]
    print(f"[RAG] After filters: {len(docs)}")

    # 검색된 문서 내용을 하나의 문자열로 합치기
    context_text = "\n\n".join(doc.page_content for doc in docs)
    # 프롬프트 + LLM 호출
    messages = prompt.invoke({"question": question, "context": context_text})
    response = llm.invoke(messages)

    return response.content, docs, applied_filters


In [11]:

print("--- Paper Chatbot ---")

if vector_store is None: #FAISS 인덱스가 로드되지 않았다면 실행할 수 없으므로 사용자에게 오류 전달
    print("[ERROR] Vector store is not loaded. Run the FAISS load cell first.")
else:
    try:
        preferences = collect_user_preferences() #사용자 선호도 입력받기
        print(f"[INFO] Preferences: {describe_preferences(preferences)}")

        user_question = input("Enter your question: ").strip()

        print(f" [You]: {user_question}")
        answer, used_docs, applied = rag_answer(user_question, k=4, preferences=preferences)

        print(f"[Bot]: {answer}")
        
        # 실제로 RAG에서 어떤 필터가 적용되었는지 사용자에게 피드백
        note = []
        if applied.get("year_cutoff"):
            note.append(f"filtered to >= {applied['year_cutoff']} year")
        vm = applied.get("venue_mode")
        if vm == "top":
            note.append("top-tier prioritized")
        elif vm == "top_only":
            note.append("top-tier only")
        elif vm == "custom":
            note.append("custom venues prioritized")
        if note:
            print("[Filters] " + " | ".join(note))
        # 사용된 문서(Sources) 출력
        print("--- Sources ---")
        for i, doc in enumerate(used_docs):
            model_name = doc.metadata.get("model_name", "Unknown")
            year = doc.metadata.get("year", "?")
            venue = doc.metadata.get("venue", "?")
            preview = doc.page_content[:120].replace("\n", " ")
            print(f"[{i+1}] {model_name} ({year}, {venue}): {preview}...")

    except Exception as e:
        import traceback
        print(f"[ERROR] RAG run failed: {e}")
        traceback.print_exc()


--- Paper Chatbot ---
[SETUP] Enter your search preferences.
[경고] 숫자가 아닙니다. 기본값 5년을 사용합니다.
[INFO] Preferences: recent 5 years | top-tier prioritized
 [You]: 고양이 발 개수는 몇개야?
[RAG] 검색 질의: 고양이 발 개수는 몇개야?
[RAG] 검색된 문서 수: 3
[RAG] After filters: 1
[Bot]: 제공된 논문/모델 정보에는 고양이 발 개수에 대한 정보가 포함되어 있지 않습니다.
[Filters] filtered to >= 2020 year | top-tier prioritized
--- Sources ---
[1] GPT-3 (2020, NeurIPS): 모델명: GPT-3. title: Language Models are Few-Shot Learners author: Tom B. Brown et al. year: 2020 venue: NeurIPS task: Lan...
